# OpenSearch Neural Sparse Model - 학습 데이터 준비

## 목표
한국어 공개 데이터셋(Wikipedia, Namuwiki)을 활용하여 OpenSearch neural sparse 모델 학습에 사용할 수 있는 JSONL 형식의 데이터셋을 생성합니다.

## 데이터 포맷
```json
{
    "query": "질문 텍스트",
    "docs": ["문서1", "문서2", "문서3", ...],
    "scores": [9.5, 7.2, 5.1, ...]
}
```

## 주요 단계
1. 데이터 로딩 및 전처리
2. Query 생성 (문서 제목 활용)
3. Embedding 생성 (intfloat/multilingual-e5-large)
4. Hard Negatives Mining (FAISS 유사도 검색)
5. K-means 클러스터링으로 관련 문서 그룹화
6. 최종 JSONL 데이터셋 생성

## 1. 환경 설정

In [ ]:
import json
import os
import sys
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import random
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.cluster import MiniBatchKMeans
import torch
from sentence_transformers import SentenceTransformer
import faiss

# 프로젝트 루트 경로
PROJECT_ROOT = Path("/home/west/Documents/cursor-workspace/opensearch-neural-pre-train")
sys.path.append(str(PROJECT_ROOT))

# 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# GPU 사용 가능 여부 확인
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# 출력 디렉토리 생성
OUTPUT_DIR = PROJECT_ROOT / "dataset" / "neural_sparse_training"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EMBEDDINGS_DIR = OUTPUT_DIR / "embeddings"
EMBEDDINGS_DIR.mkdir(exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")

## 2. 데이터 로딩 및 전처리

In [ ]:
@dataclass
class Document:
    """문서 데이터 클래스"""
    id: str
    title: str
    text: str
    url: Optional[str] = None
    source: Optional[str] = None
    
    def __post_init__(self) -> None:
        """데이터 검증 및 정제"""
        self.title = self.title.strip()
        self.text = self.text.strip()


def load_jsonl_file(file_path: Path) -> List[Dict]:
    """JSONL 파일을 로드합니다.
    
    Args:
        file_path: JSONL 파일 경로
        
    Returns:
        딕셔너리 리스트
    """
    documents = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                documents.append(json.loads(line))
    return documents


def load_korean_datasets(
    wiki_limit: Optional[int] = None,
    namuwiki_limit: Optional[int] = None,
    min_text_length: int = 100,
    max_text_length: int = 2000,
) -> List[Document]:
    """한국어 Wikipedia와 Namuwiki 데이터를 로드합니다.
    
    Args:
        wiki_limit: Wikipedia 문서 개수 제한 (None이면 전체)
        namuwiki_limit: Namuwiki 문서 개수 제한 (None이면 전체)
        min_text_length: 최소 텍스트 길이
        max_text_length: 최대 텍스트 길이
        
    Returns:
        Document 객체 리스트
    """
    documents = []
    
    # Wikipedia 데이터 로드
    wiki_dir = PROJECT_ROOT / "dataset" / "wikipedia"
    wiki_files = sorted(wiki_dir.glob("ko_articles_chunk_*.jsonl"))
    
    print(f"Loading Wikipedia data from {len(wiki_files)} files...")
    wiki_count = 0
    for file_path in tqdm(wiki_files, desc="Wikipedia"):
        for item in load_jsonl_file(file_path):
            if wiki_limit and wiki_count >= wiki_limit:
                break
                
            text = item.get("text", "")
            if min_text_length <= len(text) <= max_text_length:
                documents.append(Document(
                    id=f"wiki_{item['id']}",
                    title=item.get("title", ""),
                    text=text,
                    url=item.get("url"),
                    source="wikipedia"
                ))
                wiki_count += 1
        
        if wiki_limit and wiki_count >= wiki_limit:
            break
    
    print(f"Loaded {wiki_count} Wikipedia documents")
    
    # Namuwiki 데이터 로드
    namuwiki_dir = PROJECT_ROOT / "dataset" / "namuwiki"
    if namuwiki_dir.exists():
        namuwiki_files = sorted(namuwiki_dir.glob("namuwiki_chunk_*.jsonl"))
        
        print(f"Loading Namuwiki data from {len(namuwiki_files)} files...")
        namuwiki_count = 0
        for file_path in tqdm(namuwiki_files, desc="Namuwiki"):
            for item in load_jsonl_file(file_path):
                if namuwiki_limit and namuwiki_count >= namuwiki_limit:
                    break
                    
                text = item.get("text", "")
                if min_text_length <= len(text) <= max_text_length:
                    documents.append(Document(
                        id=f"namu_{item.get('id', namuwiki_count)}",
                        title=item.get("title", ""),
                        text=text,
                        url=item.get("url"),
                        source="namuwiki"
                    ))
                    namuwiki_count += 1
            
            if namuwiki_limit and namuwiki_count >= namuwiki_limit:
                break
        
        print(f"Loaded {namuwiki_count} Namuwiki documents")
    
    print(f"Total documents loaded: {len(documents)}")
    return documents

In [ ]:
# 데이터 로드 (처음에는 샘플로 시작)
# 전체 데이터: wiki_limit=None, namuwiki_limit=None
documents = load_korean_datasets(
    wiki_limit=10000,  # 테스트용으로 제한
    namuwiki_limit=5000,  # 테스트용으로 제한
    min_text_length=100,
    max_text_length=2000,
)

print(f"\nSample document:")
print(f"Title: {documents[0].title}")
print(f"Text length: {len(documents[0].text)}")
print(f"Text preview: {documents[0].text[:200]}...")

## 3. Query 생성

문서 제목을 query로 사용하고, 본문을 positive document로 사용합니다.

In [ ]:
@dataclass
class QueryDocPair:
    """Query-Document 쌍"""
    query_id: str
    query_text: str
    positive_doc_id: str
    positive_doc_text: str
    source: str


def create_query_doc_pairs(documents: List[Document]) -> List[QueryDocPair]:
    """문서로부터 Query-Document 쌍을 생성합니다.
    
    Args:
        documents: Document 객체 리스트
        
    Returns:
        QueryDocPair 객체 리스트
    """
    pairs = []
    
    for doc in tqdm(documents, desc="Creating query-doc pairs"):
        if not doc.title or not doc.text:
            continue
            
        pairs.append(QueryDocPair(
            query_id=f"q_{doc.id}",
            query_text=doc.title,
            positive_doc_id=doc.id,
            positive_doc_text=doc.text,
            source=doc.source,
        ))
    
    return pairs


qd_pairs = create_query_doc_pairs(documents)
print(f"Created {len(qd_pairs)} query-document pairs")

print(f"\nSample pair:")
print(f"Query: {qd_pairs[0].query_text}")
print(f"Document: {qd_pairs[0].positive_doc_text[:200]}...")

## 3.5. Query Augmentation using Ollama (LLM)

원본 query(문서 제목)를 Ollama의 LLM 모델로 증강하여 다양한 검색 쿼리를 생성합니다.
이를 통해 학습 데이터의 다양성을 높이고 모델의 robustness를 향상시킵니다.

In [ ]:
import requests
from typing import List, Dict
import time

# Ollama 설정
OLLAMA_MODEL = "qwen3:30b-a3b-instruct-2507-q8_0"
OLLAMA_API_URL = "http://localhost:11434/api/generate"

def generate_with_ollama(
    prompt: str,
    model: str = OLLAMA_MODEL,
    temperature: float = 0.7,
    max_tokens: int = 200,
) -> str:
    """Ollama API를 사용하여 텍스트를 생성합니다.
    
    Args:
        prompt: 입력 프롬프트
        model: Ollama 모델 이름
        temperature: 생성 temperature
        max_tokens: 최대 토큰 수
        
    Returns:
        생성된 텍스트
    """
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "num_predict": max_tokens,
        }
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        return result.get("response", "").strip()
    except Exception as e:
        print(f"Error calling Ollama API: {e}")
        return ""


def augment_query(
    original_query: str,
    num_variations: int = 2,
) -> List[str]:
    """LLM을 사용하여 query를 증강합니다.
    
    Args:
        original_query: 원본 query
        num_variations: 생성할 변형 query 개수
        
    Returns:
        증강된 query 리스트 (원본 포함)
    """
    prompt = f"""주어진 검색 쿼리에 대해 {num_variations}개의 다양한 변형 쿼리를 생성하세요.
변형 쿼리는 원본 쿼리와 같은 의미를 가지지만, 다른 표현 방식을 사용해야 합니다.

원본 쿼리: {original_query}

다음 형식으로 출력하세요 (번호와 쿼리만, 추가 설명 없이):
1. [변형 쿼리 1]
2. [변형 쿼리 2]
"""
    
    response = generate_with_ollama(prompt, temperature=0.8)
    
    # 응답에서 query 추출
    augmented_queries = [original_query]  # 원본 query 포함
    
    if response:
        lines = response.split('\n')
        for line in lines:
            line = line.strip()
            # "1. ", "2. " 등의 형식에서 query 추출
            if line and (line[0].isdigit() or line.startswith('-') or line.startswith('*')):
                # 번호나 bullet 제거
                query = line.split('.', 1)[-1].strip() if '.' in line else line.lstrip('-*').strip()
                # 대괄호 제거
                query = query.strip('[]')
                if query and query != original_query:
                    augmented_queries.append(query)
    
    # 정확히 num_variations + 1개 반환 (원본 + 변형들)
    return augmented_queries[:num_variations + 1]


# Ollama 연결 테스트
print("Testing Ollama connection...")
test_response = generate_with_ollama("안녕하세요", max_tokens=50)
if test_response:
    print(f"✅ Ollama connected successfully!")
    print(f"Test response: {test_response[:100]}...")
else:
    print("⚠️ Ollama connection failed. Please check if Ollama is running.")
    print(f"Model: {OLLAMA_MODEL}")
    print(f"API URL: {OLLAMA_API_URL}")

In [ ]:
# Query Augmentation 수행
# 전체 query에 대해 증강을 수행하면 시간이 오래 걸리므로 샘플링 옵션 제공
ENABLE_QUERY_AUGMENTATION = True  # False로 설정하면 증강 건너뛰기
AUGMENTATION_SAMPLE_RATE = 0.3  # 전체 query의 30%만 증강 (1.0이면 전체)
NUM_QUERY_VARIATIONS = 2  # 각 query당 생성할 변형 개수

if ENABLE_QUERY_AUGMENTATION:
    print(f"Query Augmentation Settings:")
    print(f"  Sample rate: {AUGMENTATION_SAMPLE_RATE * 100:.0f}%")
    print(f"  Variations per query: {NUM_QUERY_VARIATIONS}")
    print(f"  Total queries to augment: {int(len(qd_pairs) * AUGMENTATION_SAMPLE_RATE)}")
    
    # 증강할 query 인덱스 선택
    num_to_augment = int(len(qd_pairs) * AUGMENTATION_SAMPLE_RATE)
    augment_indices = random.sample(range(len(qd_pairs)), num_to_augment)
    
    # Query augmentation 실행
    augmented_qd_pairs = []
    failed_count = 0
    
    for i, pair in enumerate(tqdm(qd_pairs, desc="Augmenting queries")):
        # 원본 query-doc pair는 항상 포함
        augmented_qd_pairs.append(pair)
        
        # 선택된 인덱스만 증강
        if i in augment_indices:
            augmented_queries = augment_query(
                pair.query_text,
                num_variations=NUM_QUERY_VARIATIONS,
            )
            
            # 증강된 query들 추가 (원본 제외)
            for j, aug_query in enumerate(augmented_queries[1:], 1):
                if aug_query and aug_query != pair.query_text:
                    augmented_qd_pairs.append(QueryDocPair(
                        query_id=f"{pair.query_id}_aug{j}",
                        query_text=aug_query,
                        positive_doc_id=pair.positive_doc_id,
                        positive_doc_text=pair.positive_doc_text,
                        source=pair.source,
                    ))
                else:
                    failed_count += 1
            
            # API rate limiting 방지 (너무 빠르게 호출하지 않도록)
            time.sleep(0.1)
    
    print(f"\n✅ Query augmentation completed!")
    print(f"  Original queries: {len(qd_pairs)}")
    print(f"  Augmented queries: {len(augmented_qd_pairs) - len(qd_pairs)}")
    print(f"  Total queries: {len(augmented_qd_pairs)}")
    print(f"  Failed augmentations: {failed_count}")
    
    # 증강된 쌍으로 교체
    qd_pairs = augmented_qd_pairs
    
    # 샘플 출력
    print(f"\nSample augmented queries:")
    for i in range(min(3, len(augment_indices))):
        idx = augment_indices[i]
        original_query = qd_pairs[idx].query_text
        aug_queries = [p.query_text for p in qd_pairs if p.positive_doc_id == qd_pairs[idx].positive_doc_id]
        print(f"\n  Original: {original_query}")
        for j, aug_q in enumerate(aug_queries[1:], 1):
            print(f"  Variation {j}: {aug_q}")
else:
    print("⚠️ Query augmentation is disabled. Using original queries only.")

## 4. Embedding 생성 (intfloat/multilingual-e5-large)

In [ ]:
# 모델 로드
MODEL_NAME = "intfloat/multilingual-e5-large"
print(f"Loading model: {MODEL_NAME}")

model = SentenceTransformer(MODEL_NAME, device=DEVICE)
print(f"Model loaded successfully. Embedding dimension: {model.get_sentence_embedding_dimension()}")

In [ ]:
def generate_embeddings(
    texts: List[str],
    model: SentenceTransformer,
    batch_size: int = 32,
    prefix: str = "",
) -> np.ndarray:
    """텍스트 리스트에 대한 임베딩을 생성합니다.
    
    Args:
        texts: 텍스트 리스트
        model: SentenceTransformer 모델
        batch_size: 배치 크기
        prefix: E5 모델용 prefix ("query: " 또는 "passage: ")
        
    Returns:
        임베딩 배열 (N, D)
    """
    if prefix:
        texts = [f"{prefix}{text}" for text in texts]
    
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        normalize_embeddings=True,
    )
    
    return embeddings


# Query 임베딩 생성
print("Generating query embeddings...")
query_texts = [pair.query_text for pair in qd_pairs]
query_embeddings = generate_embeddings(
    query_texts,
    model,
    batch_size=32,
    prefix="query: ",  # E5 모델은 query/passage prefix 사용
)

print(f"Query embeddings shape: {query_embeddings.shape}")

# Document 임베딩 생성
print("\nGenerating document embeddings...")
doc_texts = [pair.positive_doc_text for pair in qd_pairs]
doc_embeddings = generate_embeddings(
    doc_texts,
    model,
    batch_size=32,
    prefix="passage: ",
)

print(f"Document embeddings shape: {doc_embeddings.shape}")

In [ ]:
# 임베딩 저장
np.save(EMBEDDINGS_DIR / "query_embeddings.npy", query_embeddings)
np.save(EMBEDDINGS_DIR / "document_embeddings.npy", doc_embeddings)
print(f"Embeddings saved to {EMBEDDINGS_DIR}")

## 5. Hard Negatives Mining (FAISS)

각 query에 대해 유사하지만 관련 없는 문서를 hard negatives로 선정합니다.

In [ ]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.Index:
    """FAISS 인덱스를 생성합니다.
    
    Args:
        embeddings: 임베딩 배열 (N, D)
        
    Returns:
        FAISS 인덱스
    """
    dimension = embeddings.shape[1]
    
    # L2 정규화된 벡터이므로 Inner Product = Cosine Similarity
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings.astype(np.float32))
    
    return index


# FAISS 인덱스 생성
print("Building FAISS index...")
doc_index = build_faiss_index(doc_embeddings)
print(f"FAISS index built. Total documents: {doc_index.ntotal}")

In [ ]:
def search_hard_negatives(
    query_embeddings: np.ndarray,
    doc_index: faiss.Index,
    k: int = 10,
) -> Tuple[np.ndarray, np.ndarray]:
    """각 query에 대해 유사한 문서를 검색합니다.
    
    Args:
        query_embeddings: Query 임베딩 (N, D)
        doc_index: FAISS 인덱스
        k: 검색할 문서 개수
        
    Returns:
        (distances, indices) 튜플
    """
    distances, indices = doc_index.search(
        query_embeddings.astype(np.float32),
        k,
    )
    
    return distances, indices


# Hard negatives 검색 (top-10)
print("Searching for hard negatives...")
NUM_CANDIDATES = 10  # positive 1개 + hard negatives 9개 검색

distances, indices = search_hard_negatives(
    query_embeddings,
    doc_index,
    k=NUM_CANDIDATES,
)

print(f"Search completed. Shape: {indices.shape}")
print(f"Sample distances: {distances[0]}")
print(f"Sample indices: {indices[0]}")

## 6. K-means 클러스터링

문서를 클러스터링하여 관련 문서를 그룹화하고, 클러스터 내 문서들에 대한 관련성 점수를 계산합니다.

In [ ]:
def perform_kmeans_clustering(
    embeddings: np.ndarray,
    n_clusters: int = 100,
    batch_size: int = 1000,
) -> Tuple[MiniBatchKMeans, np.ndarray]:
    """K-means 클러스터링을 수행합니다.
    
    Args:
        embeddings: 임베딩 배열 (N, D)
        n_clusters: 클러스터 개수
        batch_size: MiniBatchKMeans 배치 크기
        
    Returns:
        (kmeans 모델, cluster labels)
    """
    print(f"Performing K-means clustering with {n_clusters} clusters...")
    
    kmeans = MiniBatchKMeans(
        n_clusters=n_clusters,
        batch_size=batch_size,
        random_state=SEED,
        verbose=1,
    )
    
    cluster_labels = kmeans.fit_predict(embeddings)
    
    return kmeans, cluster_labels


# 클러스터 개수는 데이터 크기에 따라 조정
n_clusters = min(100, len(documents) // 50)
print(f"Using {n_clusters} clusters for {len(documents)} documents")

kmeans, cluster_labels = perform_kmeans_clustering(
    doc_embeddings,
    n_clusters=n_clusters,
)

print(f"Clustering completed.")
print(f"Cluster distribution: {np.bincount(cluster_labels)[:10]}...")

## 7. 최종 JSONL 데이터셋 생성

In [ ]:
@dataclass
class TrainingSample:
    """학습 샘플 데이터 클래스"""
    query: str
    docs: List[str]
    scores: List[float]


def create_training_samples(
    qd_pairs: List[QueryDocPair],
    search_indices: np.ndarray,
    search_scores: np.ndarray,
    num_docs_per_query: int = 8,
    positive_score: float = 10.0,
) -> List[TrainingSample]:
    """최종 학습 샘플을 생성합니다.
    
    Args:
        qd_pairs: Query-Document 쌍 리스트
        search_indices: FAISS 검색 결과 인덱스
        search_scores: FAISS 검색 결과 점수
        num_docs_per_query: 각 query당 문서 개수
        positive_score: Positive document 점수
        
    Returns:
        TrainingSample 리스트
    """
    samples = []
    
    for i, pair in enumerate(tqdm(qd_pairs, desc="Creating training samples")):
        docs = []
        scores = []
        
        # 1. Positive document (항상 첫 번째)
        docs.append(pair.positive_doc_text)
        scores.append(positive_score)
        
        # 2. Hard negatives (FAISS 검색 결과에서 선택)
        # 첫 번째는 자기 자신이므로 제외
        for j in range(1, min(num_docs_per_query, len(search_indices[i]))):
            neg_idx = search_indices[i][j]
            neg_score = search_scores[i][j]
            
            # 자기 자신이 아닌 경우만 추가
            if neg_idx != i:
                docs.append(qd_pairs[neg_idx].positive_doc_text)
                # Cosine similarity를 0-10 스케일로 변환
                # neg_score는 0~1 범위이므로 positive_score보다 낮게 설정
                scaled_score = float(neg_score * (positive_score - 1.0))
                scores.append(scaled_score)
        
        # 문서가 충분하지 않으면 랜덤 샘플링
        while len(docs) < num_docs_per_query:
            random_idx = random.randint(0, len(qd_pairs) - 1)
            if random_idx != i:
                docs.append(qd_pairs[random_idx].positive_doc_text)
                scores.append(0.5)  # Low score for random negatives
        
        samples.append(TrainingSample(
            query=pair.query_text,
            docs=docs[:num_docs_per_query],
            scores=scores[:num_docs_per_query],
        ))
    
    return samples


# 학습 샘플 생성
NUM_DOCS_PER_QUERY = 8  # positive 1개 + negatives 7개

training_samples = create_training_samples(
    qd_pairs,
    indices,
    distances,
    num_docs_per_query=NUM_DOCS_PER_QUERY,
)

print(f"\nCreated {len(training_samples)} training samples")
print(f"\nSample:")
sample = training_samples[0]
print(f"Query: {sample.query}")
print(f"Num docs: {len(sample.docs)}")
print(f"Scores: {sample.scores}")
print(f"First doc preview: {sample.docs[0][:100]}...")

In [ ]:
# Train/Val split
random.shuffle(training_samples)
split_idx = int(len(training_samples) * 0.9)

train_samples = training_samples[:split_idx]
val_samples = training_samples[split_idx:]

print(f"Train samples: {len(train_samples)}")
print(f"Val samples: {len(val_samples)}")

In [ ]:
def save_jsonl(
    samples: List[TrainingSample],
    file_path: Path,
) -> None:
    """학습 샘플을 JSONL 파일로 저장합니다.
    
    Args:
        samples: TrainingSample 리스트
        file_path: 출력 파일 경로
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for sample in tqdm(samples, desc=f"Saving to {file_path.name}"):
            json_obj = {
                "query": sample.query,
                "docs": sample.docs,
                "scores": sample.scores,
            }
            f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
    
    print(f"Saved {len(samples)} samples to {file_path}")


# JSONL 파일로 저장
save_jsonl(train_samples, OUTPUT_DIR / "train.jsonl")
save_jsonl(val_samples, OUTPUT_DIR / "val.jsonl")

## 8. 메타데이터 저장

In [ ]:
# 메타데이터 생성
metadata = {
    "created_at": datetime.now().isoformat(),
    "total_documents": len(documents),
    "total_queries": len(qd_pairs),
    "train_samples": len(train_samples),
    "val_samples": len(val_samples),
    "docs_per_query": NUM_DOCS_PER_QUERY,
    "embedding_model": MODEL_NAME,
    "embedding_dimension": query_embeddings.shape[1],
    "source_datasets": ["wikipedia_ko", "namuwiki"],
    "num_clusters": n_clusters,
    "min_text_length": 100,
    "max_text_length": 2000,
    "data_format": "pre-computed knowledge distillation",
    "compatible_with": "opensearch-sparse-model-tuning-sample",
}

# 메타데이터 저장
with open(OUTPUT_DIR / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print("\nMetadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

## 9. 데이터 검증

In [ ]:
# 저장된 파일 검증
print("Validating saved files...\n")

# Train 파일 검증
with open(OUTPUT_DIR / "train.jsonl", "r", encoding="utf-8") as f:
    first_line = json.loads(f.readline())
    print("Train sample:")
    print(f"  Query: {first_line['query']}")
    print(f"  Num docs: {len(first_line['docs'])}")
    print(f"  Num scores: {len(first_line['scores'])}")
    print(f"  Scores: {first_line['scores']}")
    print(f"  Max score: {max(first_line['scores'])}")
    print(f"  Min score: {min(first_line['scores'])}")

# 점수 분포 확인
all_scores = []
with open(OUTPUT_DIR / "train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line)
        all_scores.extend(sample['scores'])

all_scores = np.array(all_scores)
print(f"\nScore distribution:")
print(f"  Mean: {all_scores.mean():.2f}")
print(f"  Std: {all_scores.std():.2f}")
print(f"  Min: {all_scores.min():.2f}")
print(f"  Max: {all_scores.max():.2f}")

print(f"\n✅ Data preparation completed successfully!")
print(f"\nOutput directory: {OUTPUT_DIR}")
print(f"Files created:")
print(f"  - train.jsonl ({len(train_samples)} samples)")
print(f"  - val.jsonl ({len(val_samples)} samples)")
print(f"  - metadata.json")
print(f"  - embeddings/query_embeddings.npy")
print(f"  - embeddings/document_embeddings.npy")